## Convolutional Neural Networks

In this assignment, we will learn about convolutional neural networks. We will create a CNN and learn to classify image data.

In this lecture, we will use the image data generator to classify our data. The data is loaded below:

In [1]:
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization 
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

In [2]:
PATH = '/content/drive/MyDrive/content'
train_data_dir = PATH + '/dogs-vs-cats/train/'
test_data_dir = PATH + '/dogs-vs-cats/test/'

img_width, img_height = 150, 150
batch_size = 80

In [3]:
#This block of code is used to ensure the input shape is correct

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

Define a train data generator with shear range of 0.3, zoom range of 0.1 and rescale to 1./255 (note that we must make 1 a float to produce a correct fraction). Use the ImageDataGenerator function.

In [4]:
# Answer below:

#parameters
class_mode = 'binary'

# create generator
datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.25, 
                             shear_range = 0.3,
                              zoom_range = 0.1)


Define a test data generator that only rescales to 1./255. Use the ImageDataGenerator function.

In [5]:
# Answer below:
# create generator
testgen = ImageDataGenerator(rescale=1./255., shear_range = 0.3,
                                            zoom_range = 0.1,)

The train generator and the test generator are defined below:

In [6]:
# prepare an iterators for each dataset
train = datagen.flow_from_directory(train_data_dir, 
                                    class_mode=class_mode, 
                                    target_size =(img_width, img_height), 
                                    shuffle=True,
                                    batch_size=32,
                                    subset="training")

valid = datagen.flow_from_directory(train_data_dir,
                                            shuffle=True,
                                            class_mode=class_mode,
                                            target_size=(img_width, img_height), 
                                            batch_size=32,
                                            subset="validation")


#Shuffle off for test data so that I can run the classification report against prediction made on this data. 
test = testgen.flow_from_directory(test_data_dir, 
                                   shuffle=False,
                                   class_mode=class_mode,
                                   batch_size = 10,
                                   target_size=(img_width, img_height))
# confirm the iterator works
batchX, batchy = train.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Found 1500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Found 802 images belonging to 2 classes.
Batch shape=(32, 150, 150, 3), min=0.000, max=1.000


We'll start with a simple model. In CNNs, we first convolve the to extract features and then we add the dense layers. 

Create a model with one layer of convolution of size 64, one layer of activation, one layer of max pooling with pool size (2,2) and then one flattening layer, one dense layer of unit size 64 with a ReLU activation and one dense output layer. The output layer should have a sigmoid activation.

In [7]:
input_shape

(150, 150, 3)

In [8]:
# Answer below:

CNN_model = Sequential()

#Input Layer
CNN_model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=input_shape))
CNN_model.add(Activation('relu'))
CNN_model.add(MaxPooling2D(pool_size=(2, 2)))

#Output Layer
CNN_model.add(Flatten())
CNN_model.add(Dense(64, activation='relu'))
CNN_model.add(Dense(1, activation='sigmoid'))

In [9]:
CNN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 64)      1792      
_________________________________________________________________
activation (Activation)      (None, 150, 150, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 360000)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                23040064  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 23,041,921
Trainable params: 23,041,921
Non-trainable params: 0
____________________________________________

Compile the model using RMSprop.

In [10]:
# Answer below:
CNN_model.compile(optimizer='rmsprop',loss="binary_crossentropy",metrics=["MSE", "accuracy"])

Fit the model using a fit generator. Use 50 epochs, 25 training steps and 15 validation steps

In [11]:
EPOCHS = 50
STEP_SIZE_TRAIN = 25
STEP_SIZE_VALID = 15

# Answer below:
CNN_history = CNN_model.fit(train,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid,
                    validation_steps=STEP_SIZE_VALID,            
                    epochs = EPOCHS)
history = pd.DataFrame(CNN_history.history)
history['model'] = "One"

Epoch 1/50
25/25 [==============================] - 14s 448ms/step - loss: 8.2943 - MSE: 0.3944 - accuracy: 0.4827 - val_loss: 0.6650 - val_MSE: 0.2362 - val_accuracy: 0.5917
Epoch 2/50
25/25 [==============================] - 11s 436ms/step - loss: 0.6793 - MSE: 0.2420 - accuracy: 0.5728 - val_loss: 0.6914 - val_MSE: 0.2460 - val_accuracy: 0.6083
Epoch 3/50
25/25 [==============================] - 11s 435ms/step - loss: 0.6690 - MSE: 0.2383 - accuracy: 0.5961 - val_loss: 0.6900 - val_MSE: 0.2486 - val_accuracy: 0.5292
Epoch 4/50
25/25 [==============================] - 11s 429ms/step - loss: 0.6711 - MSE: 0.2364 - accuracy: 0.6225 - val_loss: 0.6502 - val_MSE: 0.2273 - val_accuracy: 0.6708
Epoch 5/50
25/25 [==============================] - 11s 433ms/step - loss: 0.6250 - MSE: 0.2179 - accuracy: 0.6498 - val_loss: 0.6423 - val_MSE: 0.2240 - val_accuracy: 0.6583
Epoch 6/50
25/25 [==============================] - 11s 430ms/step - loss: 0.6035 - MSE: 0.2076 - accuracy: 0.7067 - val_loss

Create a new model by adding an additional group of convolution, activation and max pooling layers before the flatten layer. Make the convolution layer of unit size 32. Keep everything else the same.

In [12]:
# Answer below:
# Answer below:

new_model = Sequential()

#Input Layer
new_model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=input_shape))
new_model.add(Activation('relu'))
new_model.add(MaxPooling2D(pool_size=(2, 2)))

#Second Convolutional layer.
new_model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=input_shape))
new_model.add(Activation('relu'))
new_model.add(MaxPooling2D(pool_size=(2, 2)))


#Output Layer
new_model.add(Flatten())
new_model.add(Dense(64, activation='relu'))
new_model.add(Dense(1, activation='sigmoid'))


Fit and compile the model in the same way you did with the previous model. How did the results improve?

In [13]:
# Answer below:
# Answer below:
new_model.compile(optimizer='rmsprop',loss="binary_crossentropy",metrics=["MSE", "accuracy"])

# Answer below:
new_history = new_model.fit(train,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid,
                    validation_steps=STEP_SIZE_VALID,            
                    epochs = EPOCHS)

new_history = pd.DataFrame(new_history.history)
new_history['model'] = 'Two'
history = pd.concat([history, new_history])

Epoch 1/50
25/25 [==============================] - 12s 465ms/step - loss: 1.1490 - MSE: 0.2816 - accuracy: 0.5204 - val_loss: 0.6919 - val_MSE: 0.2494 - val_accuracy: 0.5104
Epoch 2/50
25/25 [==============================] - 11s 436ms/step - loss: 0.7170 - MSE: 0.2595 - accuracy: 0.5345 - val_loss: 0.6859 - val_MSE: 0.2464 - val_accuracy: 0.6438
Epoch 3/50
25/25 [==============================] - 11s 439ms/step - loss: 0.6951 - MSE: 0.2487 - accuracy: 0.6092 - val_loss: 1.7247 - val_MSE: 0.4529 - val_accuracy: 0.5104
Epoch 4/50
25/25 [==============================] - 11s 433ms/step - loss: 0.7850 - MSE: 0.2635 - accuracy: 0.6358 - val_loss: 0.6487 - val_MSE: 0.2288 - val_accuracy: 0.6646
Epoch 5/50
25/25 [==============================] - 11s 434ms/step - loss: 0.7615 - MSE: 0.2451 - accuracy: 0.6367 - val_loss: 0.6563 - val_MSE: 0.2304 - val_accuracy: 0.6375
Epoch 6/50
25/25 [==============================] - 11s 428ms/step - loss: 0.6709 - MSE: 0.2363 - accuracy: 0.6230 - val_loss

It looks like there isn't an improvement. 

Create a new model based on the model above. Add an additional dense layer of size 64 with a ReLU activation after the flatten layer.

In [14]:
# Answer below:
# Answer below:
# Answer below:

new_model = Sequential()

#Input Layer
new_model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=input_shape))
new_model.add(Activation('relu'))
new_model.add(MaxPooling2D(pool_size=(2, 2)))

#Second Convolutional layer.
new_model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=input_shape))
new_model.add(Activation('relu'))
new_model.add(MaxPooling2D(pool_size=(2, 2)))



#Dense Layer
new_model.add(Flatten())
new_model.add(Dense(64, activation='relu'))
new_model.add(Dense(64, activation='relu'))

#Output Layer
new_model.add(Dense(1, activation='sigmoid'))



Fit and compile in the same way as above. Describe the difference in performance and speed.

In [15]:
# Answer below:
# Answer below:
# Answer below:
new_model.compile(optimizer='rmsprop',loss="binary_crossentropy",metrics=["MSE", "accuracy"])

# Answer below:
new_history = new_model.fit(train,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid,
                    validation_steps=STEP_SIZE_VALID,            
                    epochs = EPOCHS)

new_history = pd.DataFrame(new_history.history)
new_history['model'] = 'Three'
history = pd.concat([history, new_history])

Epoch 1/50
25/25 [==============================] - 12s 449ms/step - loss: 0.8915 - MSE: 0.2811 - accuracy: 0.4975 - val_loss: 0.6926 - val_MSE: 0.2497 - val_accuracy: 0.5271
Epoch 2/50
25/25 [==============================] - 11s 427ms/step - loss: 0.6943 - MSE: 0.2506 - accuracy: 0.4855 - val_loss: 0.6857 - val_MSE: 0.2463 - val_accuracy: 0.5104
Epoch 3/50
25/25 [==============================] - 11s 429ms/step - loss: 0.7283 - MSE: 0.2608 - accuracy: 0.5738 - val_loss: 0.6639 - val_MSE: 0.2356 - val_accuracy: 0.5854
Epoch 4/50
25/25 [==============================] - 11s 429ms/step - loss: 0.7150 - MSE: 0.2546 - accuracy: 0.5588 - val_loss: 0.7434 - val_MSE: 0.2730 - val_accuracy: 0.5063
Epoch 5/50
25/25 [==============================] - 11s 442ms/step - loss: 0.7106 - MSE: 0.2505 - accuracy: 0.5840 - val_loss: 0.6684 - val_MSE: 0.2337 - val_accuracy: 0.6500
Epoch 6/50
25/25 [==============================] - 11s 430ms/step - loss: 0.6773 - MSE: 0.2398 - accuracy: 0.6185 - val_loss

I'm still seeing an increase in my validation loss.

Fit and compile using the Adam optimizer. Describe the difference in performance between the Adam and RMSprop optimizers.

In [16]:
# Answer below:
new_model.compile(optimizer='adam',loss="binary_crossentropy",metrics=["MSE", "accuracy"])

# Answer below:
new_history = new_model.fit(train,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid,
                    validation_steps=STEP_SIZE_VALID,            
                    epochs = EPOCHS)
new_history = pd.DataFrame(new_history.history)
new_history['model'] = 'Four'
history = pd.concat([history, new_history])

Epoch 1/50
25/25 [==============================] - 12s 453ms/step - loss: 0.0614 - MSE: 0.0150 - accuracy: 0.9803 - val_loss: 2.0783 - val_MSE: 0.2790 - val_accuracy: 0.6833
Epoch 2/50
25/25 [==============================] - 11s 439ms/step - loss: 0.0443 - MSE: 0.0132 - accuracy: 0.9821 - val_loss: 2.0200 - val_MSE: 0.2986 - val_accuracy: 0.6625
Epoch 3/50
25/25 [==============================] - 11s 440ms/step - loss: 0.0291 - MSE: 0.0086 - accuracy: 0.9885 - val_loss: 2.1469 - val_MSE: 0.2874 - val_accuracy: 0.6771
Epoch 4/50
25/25 [==============================] - 11s 438ms/step - loss: 0.0360 - MSE: 0.0056 - accuracy: 0.9952 - val_loss: 2.0677 - val_MSE: 0.2868 - val_accuracy: 0.6792
Epoch 5/50
25/25 [==============================] - 11s 441ms/step - loss: 0.0293 - MSE: 0.0089 - accuracy: 0.9871 - val_loss: 2.1678 - val_MSE: 0.3072 - val_accuracy: 0.6479
Epoch 6/50
25/25 [==============================] - 11s 444ms/step - loss: 0.0190 - MSE: 0.0047 - accuracy: 0.9956 - val_loss

In [27]:
results = history.copy()

In [29]:
results.set_index('model')

,loss,MSE,accuracy,val_loss,val_MSE,val_accuracy
model,,,,,,
One,4.044320,0.355415,0.495000,0.665045,0.236179,0.591667
One,0.683487,0.241729,0.592965,0.691367,0.246047,0.608333
One,0.673814,0.239848,0.601250,0.689977,0.248601,0.529167
One,0.666562,0.234593,0.621859,0.650245,0.227253,0.670833
One,0.642137,0.220017,0.651250,0.642254,0.223983,0.658333
...,...,...,...,...,...,...
Four,0.009060,0.002266,0.997487,2.036657,0.285432,0.675000
Four,0.003281,0.000484,1.000000,1.999580,0.274591,0.691667
Four,0.003887,0.000717,1.000000,2.281620,0.275794,0.697917


In [52]:
grr = results.groupby(by='model').agg(['mean', 'min', 'max']) 

In [54]:
grr

loss                      ... val_accuracy                    
           mean       min       max  ...         mean       min       max
model                                ...                                 
Four   0.031002  0.001898  0.133832  ...     0.671917  0.635417  0.700000
One    0.380865  0.038096  4.044320  ...     0.652458  0.529167  0.693750
Three  0.277141  0.009445  0.780069  ...     0.660125  0.506250  0.710417
Two    0.300108  0.043992  0.881170  ...     0.641958  0.510417  0.681250

[4 rows x 18 columns]